![Iscte Logo](https://www.iscte-iul.pt/assets/images/logo_iscte_detailed.svg)

# Exploração do TAD Lista com implementação de listas ligadas
*Análise da complexidade de algoritmos*
---
**Relatório elaborado no âmbito da Unidade Curricular de Estruturas de Dados e Algoritmos do 1º ano da Licenciatura de Ciência de Dados**

Realizado por:
* 105289 - André Plancha - Turma CDA2
* 105289 - André Plancha - Turma CDA2
* 105289 - André Plancha - Turma CDA2
* 105220 - Tomás Ribeiro - Turma CDA2

Veja este notebook com o [nbViewer](https://nbviewer.org/github/notPlancha/Trabalho-de-Prog-2/blob/main/Fase%202.ipynb). O código total está em anexo.

# Introdução
Uma lista ligada é uma estrutura de dados que armazena elementos em uma ordem específica.
Foi implementado uma lista duplamente ligada que consiste numa lista ligada em que os elementos estão ligados de forma reciproca, ou seja, um elemento está ligado ao seguinte e o mesmo está ligado ao anterior.
E foi também implementado uma lista ligada circular onde a cabeça está ligada à cauda. Até agora, de um modo geral foram realizados alguns métodos sobre as implementações que permitem a gestão da mesma e implementamos também algoritmos de pesquisa binária e sequencial e os métodos de ordenação bubblesort e mergesort que vamos seguidamente fazer uma análise espácio-temporal.
O objetivo é tentar analisar a eficiência que cada algoritmo tem sobre como varia a sua ocupação no espaço bem como no tempo para assim poder perceber quanto tempo demora e quanto espaço ocupa a execução da pesquisa binária, o mergesort e bubblesort.
Para esse efeito iremos fazer testes ao desempenho dos algoritmos, representar graficamente as funções espaço e tempo, e descrever as observações retiradas desses mesmos gráficos. Utilizaremos este processo quer para lista ligada circular e a lista duplamente ligada.
Ao longo desta análise iremos formular hipóteses as quais iremos também comparar com os resultados empíricos obtidos.


In [ ]:
# Imports TODO remove not needed
import random

from doubly_linked_list import DoublyLinkedList
from circular_linked_list import CircularLinkedList
from base_classes import LinkedList
from Testes_ao_codigo import fromListToDll, fromListToCll
from typing import List, Tuple
from timeit import Timer
from numpy import random
from matplotlib import pyplot as plt

FileSeed = 10
maxNLists = 100
nlistsForN = 100

# Metodologia
Para analisármos a complexidade temporal empiricamente, vamos usar a biblioteca <code>timeit</code>, em conjunto com a biblioteca <code>numpy</code>, para construir as nossas listas ligadas. Iremos usar uma seed pré-defenida de forma aos nossos testes. No fim, iremos usar <code>matplotlib</code> para analisar os tempos dos vários algorítmos graficamente. Para a nossa análise assintótica, iremos demostrar as conclusões tiradas, baseadas em como construimos o nosso algoritmo.

In [ ]:
def randomLists(seed, nMax , eachNNLists):
    random.seed(seed)
    return [[random.randint(0, 10000, i) for _ in range(eachNNLists)] for i in range(nMax+1)]

def plotIt(title, xPoints, yPoints, axis = None, fill = False):
    if axis is None:
        axis = plt
    if fill:
        axis.fill(xPoints, yPoints)
    else:
        axis.scatter(xPoints, yPoints)
    axis.xlabel('N')
    axis.ylabel('Tempo (s)')
    if axis is None:
        plt.title(title)
        plt.show()
    else:
        axis.set_title(title)
def plotMethodTime(title, method):
    testArrays = randomLists(FileSeed, maxNLists, nlistsForN)
    X = []
    Y = []
    for i in range(len(testArrays)):
        X.append(len(testArrays[i]))
        LL = fromListToCll(testArrays[i])
        Y.append(Timer(lambda: method(LL)).timeit())
    plotIt(title, X, Y)

# Mergesort
O mergesort é um algoritmo de ordenação que divide um vetor em dois sub-vetores e ordena cada um deles de forma recursiva, juntando-as de forma ordenada. A complexidade temporal do mergesort quando usado num vetor é de O(n log n), onde n é o número de elementos do vetor. Isto é porque divide o vetor em duas metades e demora um tempo linear para fundir os vetores.
Numa lista ligada, a nossa hipótese é que este algoritmo não sofre alterações na sua complexidade temporal, sendo este algorítmo não parece ter vantagens temporais ao ter um vetor em vês desta estrutura de dados. Consoante o nosso código, mostrado e explicado de seguida, este parece ser o caso, em qualquer uma das listas implementadas, sendo que a lista ser circular ou duplamente ligada não dá nenhuma vantagem ou desvantagem em termos de eficiciência temporal.
Em termos de eficiência espacial, este algoritmo tem a possibilidade de não precisar de O(n) de complexidade espacial, como demonstrado por [Tatham (2017)](https://www.chiark.greenend.org.uk/~sgtatham/algorithms/listsort.html). Na nossa análise, iremos verificar se conseguimos alcançar essa conquista.

## Implementação
De seguida está demonstrado como este algoritmo foi implementado. Da forma como está implementada, achou-se desnecessário analisar as diferenças entre a lista duplamente ligada e lista circular, sendo que não há difernças na impleentação do algorítmo.

In [ ]:
#TODO put mergeSort here

O mergeSort implementado começa por encontrar o centro da nossa listaLigada. Este processo fai percorrer a nossa lista (começando na cabeça) até chegar a <code>self.size//2</code>, ou seja, a metade do tamanho da lista, dando ai o elemento do centro, ou o primeiro se o tamanho for impar. De forma a separar a lista em duas partes iguais, o algoritmo cria duas outras listas ligadas, <code>a</code> e <code>b</code>, com o elemento do centro sendo a cauda de <code>a</code> e o próximo desse a cabeça de <code>b</code>. Aqui, este vai ordenar estas duas listas, chamando mergeSort para estes também para estes ordenarem de forma recursiva também. Ao estes virem ordenados, a e b vão fundir-se em uma nova lista ligada, de forma ordenada tornando esta nova na lista final. Para tal, é implementado sortedMerge. Este vai adicionando a uma nova cabeça os elementos de cada uma das listas fornecidas de forma sequencial e ordenada.

#TODO explicar o isCircular e o IsBoudlyLinked dps de virificar como fica

## Complexidade espacial
Este algorítmo vai ter uma complexidade espacial de O(log n), sendo que estamos a criar novas listas cada vez que entramos na recursividade, estas correndo 2 * log n vezes. Sendo que estas listasligam diretamente ao nó com o valor, em vês de criá-lo novamente, apenas <code>LinkedList.size</code> vai aumentar a memória.

## Complexidade temporal
De seguida irá ser criado os dados sobre o tempo de execução do algoritmo, e analisado.

In [ ]:
plotMethodTime('MergeSort', DoublyLinkedList.mergeSort)

# Conclusão
